In [1]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Lambda
from keras.constraints import maxnorm
from keras.optimizers import adam
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.
/Users/jackholder/anaconda2/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
np.random.seed(10)

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
 
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
 
# 6. Preprocess class labels
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [4]:
import tensorflow as tf
from sklearn.utils.random import sample_without_replacement
from sklearn.utils import check_random_state
from scipy.linalg import circulant
from scipy.linalg import hadamard
from scipy.linalg import toeplitz
import random
import math
import tqdm

def gaussian_random_tensor(n_components, n_features, random_state=None):
    rng = check_random_state(random_state)
    components = rng.normal(loc=0.0,
                            scale=1.0 / np.sqrt(n_components),
                            size=(n_components, n_features))
    return (tf.convert_to_tensor(components, dtype = 'float32'))

def hadamard_mat(n_components, n_features):
    had = hadamard(n_features, dtype=complex)
    had = had[np.random.choice(had.shape[0], n_components, replace=False)]
    return had

def diagonal_mat(n_components, n_features):
    components = np.random.rand(n_features)
    components = np.diag(components)
    components = components[0:n_components]
    components = np.sign(components)
    return components

def circulant_random_tensor(n_components, n_features):
    components = np.random.normal(0, 1, (1, n_features))
    components = circulant(components)
    diag1 = diagonal_mat(n_features, n_features)
    # diag2 = diagonal_mat(n_features, n_features)
    # had = hadamard_mat(n_features, n_features)
    # HD = np.dot(had, diag1)
    # DHD = np.dot(diag2, HD)
    # components = np.dot(components, DHD) # Extended Ψ-regular hashing
    components = np.dot(components, diag1) # Short Ψ-regular hashing 
    components = components[np.random.choice(components.shape[0], n_components, replace=False)]
    return (tf.convert_to_tensor(components, dtype = 'float32'))

def toeplitz_random_tensor(n_components, n_features):
    components = np.random.normal(0, 1, (1, n_features))
    components = toeplitz(components)
    diag1 = diagonal_mat(n_features, n_features)
    # diag2 = diagonal_mat(n_features, n_features)
    # had = hadamard_mat(n_features, n_features)
    # HD = np.dot(had, diag1)
    # DHD = np.dot(diag2, HD)
    # components = np.dot(components, DHD) # Extended Ψ-regular hashing
    components = np.dot(components, diag1) # Short Ψ-regular hashing 
    components = components[np.random.choice(components.shape[0], n_components, replace=False)]
    return (tf.convert_to_tensor(components, dtype = 'float32'))

def kac_matrix(dim, n):
    maxiter = n * math.log(n)
    for k in tqdm.tqdm(range(1, int(maxiter))):
        i = random.randint(0, n-1)
        j = random.randint(0, n-1)
        while j == i:
            j = random.randint(0, n-1)
        theta = np.random.uniform(0, 2*math.pi)
        B_new = np.identity(n)
        B_new[i, i] = math.sin(theta)
        B_new[i, j] = math.cos(theta)
        B_new[j, i] = -math.cos(theta)
        B_new[j, j] = math.sin(theta)
        if k == 1:
            M = B_new
        else:
            M = np.dot(M, B_new)
    M = M[np.random.choice(M.shape[0], dim, replace=False)]
    return (tf.convert_to_tensor(M, dtype = 'float32'))
          
def project(x, Y):  
    features = K.int_shape(x)[1] 
    x = tf.transpose(x)
    # Y = tf.sign(Y) # use this if it is doing the sign BEFORE (we think right, he supposedly said wrong)
    X_new = K.dot(Y, x)
    X_new = tf.transpose(X_new)
    X_new = tf.sign(X_new) # use this if it is doing the sign AFTER (we think wrong, he says right)
    return X_new

In [6]:
dims = [1568, 784, 392, 196, 98, 49]
import time
accuracy = []
size = []
times = []
epochs = 10

for d in dims:
    start = time.time()
    
    if d != 1568:
        # Y = gaussian_random_tensor(d, 1568) 
        # Y = kac_matrix(d, 1568) 
        # Y = circulant_random_tensor(d, 1568) 
        Y = toeplitz_random_tensor(d, 1568)

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(1,28,28), kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    if d != 1568:
        model.add(Lambda (lambda x: project(x, Y)))
        
    model.add(Dense(50, activation='relu', kernel_constraint=maxnorm(3)))
    
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    h = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=epochs, verbose=1)
    
    a = model.evaluate(X_test, Y_test, verbose=0)[1]
    end = time.time()
    t = end - start
    accuracy.append(a)
    size.append(d)
    times.append(t)
    
import pandas as pd
df = pd.DataFrame({'Size': size, 'Accuracy': accuracy, 'Runtime': times})


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 312s - loss: 0.1548 - acc: 0.9532 - val_loss: 0.0553 - val_acc: 0.9816
Epoch 2/10
60000/60000 [==============================] - 273s - loss: 0.0511 - acc: 0.9843 - val_loss: 0.0362 - val_acc: 0.9882
Epoch 3/10
60000/60000 [==============================] - 249s - loss: 0.0351 - acc: 0.9889 - val_loss: 0.0524 - val_acc: 0.9831
Epoch 4/10
60000/60000 [==============================] - 249s - loss: 0.0293 - acc: 0.9906 - val_loss: 0.0342 - val_acc: 0.9890
Epoch 5/10
60000/60000 [==============================] - 247s - loss: 0.0224 - acc: 0.9930 - val_loss: 0.0342 - val_acc: 0.9893
Epoch 6/10
60000/60000 [==============================] - 244s - loss: 0.0194 - acc: 0.9941 - val_loss: 0.0342 - val_acc: 0.9889
Epoch 7/10
60000/60000 [==============================] - 244s - loss: 0.0177 - acc: 0.9943 - val_loss: 0.0265 - val_acc: 0.9910
Epoch 8/10
60000/60000 [=======================

In [9]:
df['time_saved'] = (max(df['Runtime']) - df['Runtime']) / max(df['Runtime'])
df['acc_lost'] = (max(df['Accuracy']) - df['Accuracy'])
df.head()


,Accuracy,Runtime,Size,time_saved,acc_lost
0,0.9897,2573.750125,1568,0.000000,0.0000
1,0.9514,2514.608601,784,0.022979,0.0383
2,0.9333,2449.671193,392,0.048209,0.0564
3,0.9363,2414.666112,196,0.061810,0.0534
4,0.8831,2395.492377,98,0.069260,0.1066


In [8]:
df.to_csv('Mnist_toep_sign_+baseline.csv')